In [1]:
import QuantLib as ql

In [2]:
# option data
maturity_date = ql.Date(15, 1, 2016)
spot_price = 127.62
strike_price = 130
volatility = 0.20 # the historical vols for a year
dividend_rate =  0.0163
option_type = ql.Option.Call

risk_free_rate = 0.001
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

calculation_date = ql.Date(8, 5, 2015)
ql.Settings.instance().evaluationDate = calculation_date

In [3]:
# construct the European Option
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

In [4]:
spot_handle = ql.QuoteHandle(
    ql.SimpleQuote(spot_price)
)
flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count)
)
flat_ts.enableExtrapolation()
dividend_yield = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_rate, day_count)
)
flat_vol_ts = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
)
bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                           dividend_yield, 
                                           flat_ts, 
                                           flat_vol_ts)

In [5]:
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()
print ("The theoretical price is ", bs_price)

The theoretical price is  6.749271812460607


In [6]:
foreign_rate = 0.003
flat_foreign_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, foreign_rate, day_count)
)
flat_foreign_ts.enableExtrapolation()

In [7]:
gk_process = ql.GarmanKohlagenProcess(spot_handle, flat_foreign_ts, flat_ts, flat_vol_ts)

In [8]:
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(gk_process))
bs_price = european_option.NPV()
print ("The theoretical price is ", bs_price)

The theoretical price is  7.301622529130837


In [9]:
barrier_type = ql.Barrier.DownOut
barrier = 1.0 # worthless barrier
rebate = 0.
barrier_option = ql.BarrierOption(barrier_type, barrier, rebate, payoff, exercise)

In [10]:
barrier_option.setPricingEngine(ql.AnalyticBarrierEngine(gk_process))
bs_price = barrier_option.NPV()
print ("The theoretical price is ", bs_price)

The theoretical price is  7.301622529130846


In [11]:
barrier_type = ql.Barrier.DownOut
barrier = 123.0
rebate = 0.
barrier_option = ql.BarrierOption(barrier_type, barrier, rebate, payoff, exercise)

In [12]:
barrier_option.setPricingEngine(ql.AnalyticBarrierEngine(gk_process))
bs_price = barrier_option.NPV()
print ("The theoretical price is ", bs_price)

The theoretical price is  3.46880770341172


In [13]:
maturity_date-calculation_date

252

In [14]:
ql.DeltaVolQuote.Spot

0

In [15]:
#atm_vol = ql.DeltaVolQuoteHandle(ql.DeltaVolQuote(0.,ql.QuoteHandle(ql.SimpleQuote(volatility)),maturity_date-calculation_date,ql.DeltaVolQuote.AtmFwd))
atm_vol = ql.DeltaVolQuoteHandle(ql.DeltaVolQuote(ql.QuoteHandle(ql.SimpleQuote(volatility)),ql.DeltaVolQuote.Fwd, maturity_date-calculation_date,ql.DeltaVolQuote.AtmFwd))
vol_25put = ql.DeltaVolQuoteHandle(ql.DeltaVolQuote(-0.25,ql.QuoteHandle(ql.SimpleQuote(volatility+0.01)),maturity_date-calculation_date,ql.DeltaVolQuote.Fwd))
vol_25call = ql.DeltaVolQuoteHandle(ql.DeltaVolQuote(0.25,ql.QuoteHandle(ql.SimpleQuote(volatility+0.009)),maturity_date-calculation_date,ql.DeltaVolQuote.Fwd))

In [17]:
barrier_option.setPricingEngine(ql.VannaVolgaBarrierEngine(atm_vol, vol_25put, vol_25call, spot_handle, flat_ts, flat_foreign_ts))
vanna_volga_price = barrier_option.NPV()
print ("The theoretical price is ", vanna_volga_price)

The theoretical price is  3.4712700775883096
